In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # remove when finished
from implementations import *
from tools import *
from helpers import *

In [2]:
data_tr = pd.read_csv('data/train.csv')
data_te = pd.read_csv('data/test.csv')
idx_tr = data_tr.Id
idx_te = data_te.Id


In [3]:
features_names = data_tr.columns.drop(['Id','Prediction'])

In [4]:

# replacing the -999 with mode in signal or background

DER_mode_s = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].value_counts().index[0]#value_counts().index[0]
DER_mode_b = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].value_counts().index[0]#mean(axis=0)

idx_s = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].index
idx_b = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].index

data_tr.loc[idx_s,'DER_mass_MMC']=DER_mode_s
data_tr.loc[idx_b,'DER_mass_MMC']=DER_mode_b

In [5]:
DER_mode_te = data_te.loc[(data_te['DER_mass_MMC']!=-999)]['DER_mass_MMC'].value_counts().index[0]#mean(axis=0)
idx_te = data_te.loc[(data_te['DER_mass_MMC']==-999)]['DER_mass_MMC'].index
data_te.loc[idx_te,'DER_mass_MMC']=DER_mode_te

In [7]:
idx_te.shape

(86488,)

In [ ]:
def group_features_by_jet(df):
    return {  
        0: df.loc[df['PRI_jet_num'] == 0].copy(),
        1: df.loc[df['PRI_jet_num'] == 1].copy(),
        2: df.loc[(df['PRI_jet_num'] == 2) | (df['PRI_jet_num'] == 3)].copy(),
    }

In [ ]:
feature_tr_grp = group_features_by_jet(data_tr)
feature_te_grp = group_features_by_jet(data_te)

In [ ]:
drop_col = {'names_70': ['DER_deltaeta_jet_jet',
  'DER_mass_jet_jet',
  'DER_prodeta_jet_jet',
  'DER_lep_eta_centrality',
  'PRI_jet_subleading_pt',
  'PRI_jet_subleading_eta',
  'PRI_jet_subleading_phi'],
 'names_30': ['PRI_jet_leading_pt',
  'PRI_jet_leading_eta',
  'PRI_jet_leading_phi'],
 'names_phi': ['PRI_jet_leading_phi',
  'PRI_jet_subleading_phi',
  'PRI_lep_phi',
  'PRI_met_phi',
  'PRI_tau_phi',
  'PRI_jet_num'],
 'corr0': ['DER_mass_MMC', 'DER_pt_tot', 'PRI_tau_pt', 'PRI_lep_pt'],
 'corr1_2': ['DER_mass_MMC',
  'DER_pt_h',
  'DER_sum_pt',
  'PRI_met_sumet',
  'PRI_jet_all_pt',
  'PRI_lep_pt']}

In [ ]:
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['names_30'])))
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = 'PRI_jet_all_pt')# all 0s
feature_tr_grp[1] = feature_tr_grp[1].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70'])))
feature_tr_grp[2] = feature_tr_grp[2].drop(columns = list(set(drop_col['names_phi'])))

feature_te_grp[0] = feature_te_grp[0].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['names_30'])))
feature_te_grp[0] = feature_te_grp[0].drop(columns = 'PRI_jet_all_pt')# all 0s
feature_te_grp[1] = feature_te_grp[1].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70'])))
feature_te_grp[2] = feature_te_grp[2].drop(columns = list(set(drop_col['names_phi'])))

In [ ]:
a = feature_te_grp[0].drop(columns =['Id','Prediction'])
a.shape

In [ ]:
a

In [ ]:
feature_te_grp[1].drop(columns =['Id','Prediction'])

In [ ]:
# ridge regression
w = []
acc_tr = []
acc_te = []
lambda_ = 0.001
degrees = [11,10,11]
k_fold = 10
seed = 10
y_te = []

for i in range(len(feature_tr_grp)):
    x = feature_tr_grp[i].drop(columns = ['Id','Prediction']).values
    x = log_process(x)
    x,_,_ = standardize(x)
    x_tr = build_poly(x,degrees[i])
    y_tr = feature_tr_grp[i].Prediction.replace(['s','b'],[1,-1]).values
    w_tmp,acc_tr_tmp,acc_te_tmp = cv_loop(y_tr, x_tr,k_fold,seed,ridge_regression,lambda_=lambda_)
    w.append(w_tmp)
    acc_tr.append(acc_tr_tmp)
    acc_te.append(acc_te_tmp)
    
    x = feature_te_grp[i].drop(columns = ['Id','Prediction']).values
    x = log_process(x)
    x,_,_ = standardize(x)
    x_te = build_poly(x,degrees[i])
    # x_te = np.hstack((np.ones((x_te.shape[0], 1)), x_te))
    y_te.append(predict_labels(w[i], x_te))

In [ ]:
acc_tr

In [ ]:
y_pred_te = np.concatenate((y_te[0],y_te[1],y_te[2]))
idx_te = np.concatenate((feature_te_grp[0].Id.values,feature_te_grp[1].Id.values,feature_te_grp[2].Id.values))

In [ ]:
create_csv_submission(idx_te, y_pred_te, 'submission_ridge.csv')

In [ ]:
idx_te

In [ ]:
 y_pred_te.shape